## RankForecasting-stint-paper

baseed on: 18/stint_test

stint rank prediction for paper



In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from indycar.model.ZeroPredictor import ZeroPredictor
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
#import indycar.model.stint_simulator as stint
import indycar.model.stint_simulator_shortterm_pitmodel as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [73]:
def simulation(datasetid, testevent, taskid, runts, expid, predictionlen, 
               datamode, loopcnt, featuremode = stint.FEATURE_STATUS,
              pitmodel = 0, model = 'oracle', inlapmode=0):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    
    stint._inlap_status = inlapmode
    
    stint.init(pitmodel)
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    predictor = stint.load_model(predictionlen, model,trainid='2018')

    ret2 = {}
    
    #the simplest case, trim=0
    stint._trim = 0
    #stint._debug_carlist=[6]
    stint._debug_carlist=[12]
    for i in range(loopcnt):
        #df, full_samples, full_tss
        ret2[i] = stint.run_simulation_pred(predictor, predictionlen, stint.freq, datamode=datamode)

    acc = []
    for i in ret2.keys():
        df = ret2[i]
        _x = stint.get_evalret(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    #save keys
    #stint._pitmodel.save_keys('pitmodel-keys.pickle')
    
    return b, ret2

def test_pitmodelx(modelfile, testevent, runts, loopcnt=10, model='oracle'):
    #pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
    if runts == 'RANK':
        acc, ret = simulation('indy2013-2018-nocarid-context40', testevent, 
                    'rank',stint.COL_RANK,'rank',
                   2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                      pitmodel=modelfile, model=model)        
    else:
        acc, ret = simulation('indy2013-2018-nocarid-context40', testevent, 
                        'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                       2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                          pitmodel=modelfile, model=model)

    #print(np.mean(acc, axis=0))
    return acc, ret

def test_pitmodel(modelfile, testevent, runts, loopcnt=10, model='oracle', inlapmode=0,featuremode = stint.FEATURE_PITAGE):
    #pitmodel-m60-mlp-e500-l10-10-5-student-d0.1.pickle
    
    lapmode = {0:'noinlap', 1:'inlap',2:'outlap'}
    
    #dataset = 'indy2013-2018%s-nocarid-context40'%('' if inlapmode==0 else '-inlapstatus')
    dataset = 'indy2013-2018-%s-%s-nocarid-context40'%(
                lapmode[inlapmode],
                'pitage' if featuremode==stint.FEATURE_PITAGE else 'nopitage'
                )
    try:
        if runts == 'RANK':
            acc, ret = simulation(dataset, testevent, 
                        'rank',stint.COL_RANK,'rank',
                       2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                          pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode)        
        else:
            acc, ret = simulation(dataset, testevent, 
                            'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                           2, stint.MODE_ORACLE_LAPONLY,loopcnt, 
                              pitmodel=modelfile, model=model, inlapmode=inlapmode,featuremode =featuremode)
    except:
        acc, ret = [],[]
        
    #print(np.mean(acc, axis=0))
    return acc, ret

In [3]:
acc, ret, pret = {}, {}, {}

In [4]:
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}
for runts in ['TIMEDIFF','RANK']:
    for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
        for year in ['2018','2019']:
            for lapmode in [0,1,2]:
                mid = 'oracle-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
                acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-%s'%year, runts, loopcnt=1,inlapmode=lapmode,featuremode = fmode)
                #if acc[mid] != []:
                #    pret[mid] = prisk(ret[mid][0][1], ret[mid][0][2])
                #else:
                #    pret[mid] = []


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
model: acc={0.79}, mae={3.11}, rmse={5.10},r2={0.37}, 
            naive: acc={0.17}, mae={5.72}, rmse={8.08},r2={0.29}
[[0.79130435 3.11304348 5.09503879 0.3655601 ]
 [0.17391304 5.72173913 8.08111056 0.28710352]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
model: acc={0.80}, mae={3.09}, rmse={5.11},r2={0.35}, 
            naive: acc={0.17}, mae={5.72}, rmse={8.08},r2={0.29}
[[0.8        3.08985507 5.11264417 0.34830313]
 [0.17391304 5.72173913 8.08111056 0.28710352]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


...

init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a 

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a 

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a 

In [5]:
fmodestr = {stint.FEATURE_PITAGE:'pitage',stint.FEATURE_STATUS:'nopitage'}
lapstr = {0:'noinlap',1:'inlap',2:'outlap'}
for runts in ['TIMEDIFF','RANK']:
    for fmode in [stint.FEATURE_PITAGE, stint.FEATURE_STATUS]:
        for year in ['2018','2019']:
            for lapmode in [0,1,2]:
                mid = 'pitmodel-%s-%s-%s-%s'%(runts, year, lapstr[lapmode], fmodestr[fmode])
                acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 
                                    'Indy500-%s'%year, runts, loopcnt=3,inlapmode=lapmode,featuremode = fmode)
                #if acc[mid] != []:
                #    pret[mid] = prisk(ret[mid][0][1], ret[mid][0][2])
                #else:
                #    pret[mid] = []

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
model: acc={0.56}, mae={5.22}, rmse={6.98},r2={-0.84}, 
            naive: acc={0.15}, mae={6.02}, rmse={8.31},r2={0.25}
model: acc={0.60}, mae={5.39}, rmse={7.22},r2={-1.19}, 
            naive: acc={0.15}, mae={6.01}, rmse={8.30},r2={0.26}
model: acc={0.57}, mae={5.34}, rmse={7.19},r2={-1.08}, 
            naive: acc={0.16}, mae={5.99}, rmse={8.29},r2={0.26}
[[ 0.57854534  5.3130495   7.12901064 -1.03537489]
 [ 0.15305408  6.01025008  8.29940648  0.25591416]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
ERROR: key 14-45 not found in model
model: acc={0.52}, mae={5.95}, rmse={7.89},r2={-1.22}, 
            naive: acc={0.15}, mae={6.02}, rmse={8.31},r2={0.25}
model: acc={0.51}, mae={5.80}, rmse={7.66},r2={-1.07}, 
            naive: acc={0.16}, mae={5.99}, rmse={8.29},r2={0.26}
model: acc={0.50}, mae={5.95}, rmse={7.93},r2={-1.73}, 
            naive: acc={0.16}, mae={5.99}, rmse={8.29},r2={0.26}
[[ 0.50665729  5.90025641  7.82564247 -1.33943334]
 [ 0.15391495  6.00414009  8.29518625  0.25596508]]


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


...

init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a sh

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a sh

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a short ts: carid=88，len=4
a sh

In [6]:
acc

{'oracle-TIMEDIFF-2018-noinlap-pitage': array([[[0.79130435, 3.11304348, 5.09503879, 0.3655601 ],
         [0.17391304, 5.72173913, 8.08111056, 0.28710352]]]),
 'oracle-TIMEDIFF-2018-inlap-pitage': array([[[0.8       , 3.08985507, 5.11264417, 0.34830313],
         [0.17391304, 5.72173913, 8.08111056, 0.28710352]]]),
 'oracle-TIMEDIFF-2018-outlap-pitage': [],
 'oracle-TIMEDIFF-2019-noinlap-pitage': array([[[0.71867008, 3.4168798 , 5.62375306, 0.09946494],
         [0.20971867, 5.40409207, 7.76953853, 0.31746095]]]),
 'oracle-TIMEDIFF-2019-inlap-pitage': array([[[0.75191816, 3.08184143, 5.40696718, 0.20236806],
         [0.20971867, 5.40409207, 7.76953853, 0.31746095]]]),
 'oracle-TIMEDIFF-2019-outlap-pitage': [],
 'oracle-TIMEDIFF-2018-noinlap-nopitage': array([[[0.7826087 , 4.15652174, 6.32547186, 0.36219144],
         [0.17391304, 5.72173913, 8.08111056, 0.28710352]]]),
 'oracle-TIMEDIFF-2018-inlap-nopitage': array([[[0.8057971 , 3.14202899, 5.18246766, 0.42277607],
         [0.173913

In [7]:
sys.exit(0)

SystemExit: 0

/scratch_ssd/hpda/anaconda3/envs/gluonts/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
mid = 'oracle-2018'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2018', 'TIMEDIFF', loopcnt=1)
#_ = prisk(ret[mid][0][1], ret[mid][0][2])

#_ = prisk(ret[mid][0][1], ret[mid][0][2])

In [ ]:
mid = 'oracle-2018-rank'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2018', 'RANK', loopcnt=1)

In [ ]:
mid = 'ranknet-2018-debug'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 
                                   'Indy500-2018', 'TIMEDIFF', loopcnt=1)


In [ ]:
mid = 'ranknet-2018-rank-debug'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 
                                   'Indy500-2018', 'RANK', loopcnt=1)

In [ ]:
mid = 'oracle-2019-timediff'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2019', 'TIMEDIFF', loopcnt=1)

In [ ]:
mid = 'oracle-2019-rank'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2019', 'RANK', loopcnt=1)

In [ ]:
mid = 'oracle-2018'
acc[mid], ret[mid] = test_pitmodel('oracle', 'Indy500-2018', 'RANK', loopcnt=1)

In [ ]:
mid = 'ranknet-2018'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'RANK')


In [ ]:
mid = 'ranknet-2019'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'RANK')

In [ ]:
mid = 'ranknet-timediff-pit0-2018'
acc[mid], ret[mid] = test_pitmodel(0, 'Indy500-2018', 'TIMEDIFF',loopcnt=1)


In [ ]:
mid = 'ranknet-timediff-2018'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'TIMEDIFF',loopcnt=1)



In [ ]:
mid = 'ranknet-timediff-2019'
acc[mid], ret[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2019', 'TIMEDIFF',loopcnt=1)



In [ ]:
#
# configurataiona
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2018'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')

In [ ]:
df_oracle = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

In [ ]:
df_oracle[df_oracle['carno']==12]

In [ ]:
stint.get_evalret(df_oracle)

### test

In [ ]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2019'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')
df = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)
stint.get_evalret(df)


In [ ]:
allpits, pitmat, maxlap = stint.get_pitlaps(verbose=True)
#print(len(allpits), maxlap)
nextpit, maxnext = stint.get_nextpit(pitmat, 30)

print(nextpit, maxnext)

In [ ]:
len(pitmat)

In [ ]:
pitmat

In [ ]:
df

In [ ]:
df[df['carno']==12]

In [ ]:
correct = df[df['sign']==df['pred_sign']] 
acc = len(correct)/len(df)
acc

In [ ]:
stint.get_evalret(df)

In [ ]:
ret ={ }
ret['oracle'] = df
ret['curtrack'] = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_TESTCURTRACK )

In [ ]:
stint.get_evalret(ret['curtrack'])

In [ ]:
ret['zerotrack'] = stint.run_simulation(predictor, 2, stint.freq, datamode=stint.MODE_TESTZERO )

In [ ]:
stint.get_evalret(ret['zerotrack'])

### test sim by pred

In [ ]:
def simulation2(datasetid, testevent, taskid, runts, expid, predictionlen, datamode, loopcnt, featuremode = stint.FEATURE_STATUS):
    #
    # configurataion
    #
    # model path:  <_dataset_id>/<_task_id>-<trainid>/
    #_dataset_id = 'indy2013-2018-nocarid'
    stint.init()
    stint._dataset_id = datasetid
    stint._test_event = testevent
    #_test_event = 'Indy500-2019'

    stint._feature_mode = featuremode
    stint._context_ratio = 0.

    stint._task_id = taskid  # rank,laptime, the trained model's task
    stint._run_ts = runts   #COL_LAPTIME,COL_RANK
    stint._exp_id=expid  #rank, laptime, laptim2rank, timediff2rank... 

    stint._train_len = 40
    predictor = stint.load_model(predictionlen, 'oracle',trainid='2018')

    ret2 = {}
    for i in range(loopcnt):
        ret2[i] = stint.run_simulation_pred(predictor, predictionlen, stint.freq, datamode=datamode)

    acc = []
    for i in ret2.keys():
        df = ret2[i]
        _x = stint.get_evalret(df)
        acc.append(_x)

    b = np.array(acc)
    print(np.mean(b, axis=0))
    
    return b

In [ ]:
accret = simulation('indy2013-2018-nocarid', 'Indy500-2018', 
                    'rank',stint.COL_RANK,'rank',
                   2, stint.MODE_ORACLE_LAPONLY, 3)

In [ ]:
accret = simulation('indy2013-2018-nocarid', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,3)


In [ ]:
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

In [ ]:
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE,5)
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_TESTCURTRACK,5)
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_TESTZERO,5)

In [ ]:
accret = simulation('indy2013-2018-pitage-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5, featuremode = stint.FEATURE_PITAGE)

In [ ]:
accret = simulation('indy2013-2018', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

In [ ]:
accret = simulation('indy2013-2018', 'Indy500-2019', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

In [ ]:
### pitmodel -- top8
accret = simulation('indy2013-2018', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)
accret = simulation('indy2013-2018', 'Indy500-2019', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

In [ ]:
### pitmodel -- top8
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2018', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)
accret = simulation('indy2013-2018-nocarid-context40', 'Indy500-2019', 
                    'timediff',stint.COL_TIMEDIFF,'timediff2rank',
                   2, stint.MODE_ORACLE_LAPONLY,5)

### sim pred

In [ ]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()

#stint._dataset_id = 'indy2013-2018'
stint._test_event = 'Indy500-2018'
#_test_event = 'Indy500-2019'

#stint._dataset_id = 'indy2013-2018-nocarid-context40'
#stint._feature_mode = stint.FEATURE_STATUS
stint._dataset_id = 'indy2013-2018-pitage-nocarid-context40'
stint._feature_mode = stint.FEATURE_PITAGE

stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')
df2 = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE_LAPONLY)
stint.get_evalret(df2)
df3 = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)
stint.get_evalret(df3)

In [ ]:
df2[df2['carno']==12]

In [ ]:
df2

In [ ]:
stint.get_evalret(df2)

In [ ]:
ret2 = {}
for i in range(10):
    ret2[i] = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE_LAPONLY)
    
acc = []
for i in ret2.keys():
    df = ret2[i]
    _x = stint.get_evalret(df)
    acc.append(_x)

b = np.array(acc)
np.mean(b, axis=0)

In [ ]:
df = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

In [ ]:
df[df['carno']==12]

In [ ]:
stint.get_evalret(df)

In [ ]:
acc = []
for i in ret.keys():
    df = ret[i]
    _x = stint.get_evalret(df)
    acc.append(_x)

b = np.array(acc)
np.mean(b, axis=0)

In [ ]:
a = [(1,2,3),(2,3,4)]
np.array(a)

In [ ]:
len(ret)

In [ ]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2019'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')

In [ ]:
df = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

In [ ]:
stint.get_evalret(df)

In [ ]:
ret2 = {}
for i in range(10):
    ret2[i] = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

In [ ]:
acc = []
for i in ret2.keys():
    df = ret2[i]
    _x = stint.get_evalret(df)
    acc.append(_x)

b = np.array(acc)
np.mean(b, axis=0)

### debug

In [ ]:
#
# configurataion
#
# model path:  <_dataset_id>/<_task_id>-<trainid>/
#_dataset_id = 'indy2013-2018-nocarid'
stint.init()
stint._dataset_id = 'indy2013-2018-nocarid-context40'
stint._test_event = 'Indy500-2019'
#_test_event = 'Indy500-2019'

stint._feature_mode = stint.FEATURE_STATUS
stint._context_ratio = 0.

stint._task_id = 'timediff'  # rank,laptime, the trained model's task
stint._run_ts = stint.COL_TIMEDIFF   #COL_LAPTIME,COL_RANK
stint._exp_id='timediff2rank'  #rank, laptime, laptim2rank, timediff2rank... 

stint._train_len = 40
predictor = stint.load_model(2, 'oracle',trainid='2018')

In [ ]:
df = stint.run_simulation_pred(predictor, 2, stint.freq, datamode=stint.MODE_ORACLE)

In [ ]:
stint.get_evalret(df)

### difference ? oracle .vs. pred

In [ ]:
#df, df2
df

In [ ]:
df2

In [ ]:
oracle = df[['carno','startlap']].values
pred =  df2[['carno','startlap']].values

In [ ]:
a = set(['%s-%s'%(x[0],x[1]) for x in oracle])
b = set(['%s-%s'%(x[0],x[1]) for x in  pred])
a -b

In [ ]:
df[df['carno']==3]

In [ ]:
df2[df2['carno']==3]

In [ ]:
_, data2018, _ = stint.load_data('Indy500-2018')
data2018[data2018['completed_laps']==200]

In [ ]:
topK = data2018[data2018['completed_laps']==200]['car_number'].values
topK

In [ ]:
topK_df = df[df['carno'].isin(topK[:5])]
stint.get_evalret(topK_df)

In [ ]:
len(topK_df)

In [ ]:
topK_df2 = df2[df2['carno'].isin(topK[:5])]
stint.get_evalret(topK_df2)

In [ ]:
len(topK_df2)

### check the dataset

In [ ]:
df = ret['oracle-2018']

In [ ]:
df = df[0]
df[df['carno']==6]

In [ ]:
len(df)

In [ ]:
dftemp = df[['carno','startlap', 'startrank','endrank']].sort_values(['carno','startlap'])
test = dftemp[['carno','startlap', 'startrank','endrank']].values.astype(int)
dfout = pd.DataFrame(test, columns =['carno','startlap', 'startrank','endrank'])
dfout.to_csv('test_db_ranknet2.csv')

In [ ]:
df = ret['ranknet-2018'][0]
df[df['carno']==6]

In [ ]:
len(df)

In [ ]:
ret['ranknet-timediff-2019']

In [ ]:
ret.keys()

In [ ]:
df=ret['oracle-2018'][0]
df.to_csv('result-stint')

In [10]:
df = ret['oracle-TIMEDIFF-2018-noinlap-pitage'][0]
df[df['carno']==12]

In [11]:
df

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,13,27,11.0,20.0,9.0,1,20.0,9.0,1
1,13,28,20.0,12.0,-8.0,-1,13.0,-7.0,-1
2,14,28,5.0,10.0,5.0,1,10.0,5.0,1
3,59,28,32.0,32.0,0.0,0,32.0,0.0,0
4,3,29,6.0,10.0,4.0,1,8.0,2.0,1
...,...,...,...,...,...,...,...,...,...
340,25,194,0.0,16.0,16.0,1,16.0,16.0,1
341,59,194,20.0,21.0,1.0,1,21.0,1.0,1
342,60,194,1.0,17.0,16.0,1,17.0,16.0,1
343,19,195,9.0,15.0,6.0,1,16.0,7.0,1


In [12]:
df[df['carno']==12]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
15,12,30,1.0,3.0,2.0,1,2.0,1.0,1
30,12,31,3.0,5.0,2.0,1,2.0,-1.0,-1
71,12,48,5.0,5.0,0.0,0,5.0,0.0,0
92,12,49,5.0,0.0,-5.0,-1,0.0,-5.0,-1
169,12,92,0.0,0.0,0.0,0,0.0,0.0,0
177,12,93,0.0,0.0,0.0,0,0.0,0.0,0
235,12,127,0.0,1.0,1.0,1,0.0,0.0,0
240,12,128,1.0,0.0,-1.0,-1,0.0,-1.0,-1
309,12,169,0.0,7.0,7.0,1,3.0,3.0,1


In [13]:
df = ret['oracle-TIMEDIFF-2018-inlap-pitage'][0]
df[df['carno']==12]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
15,12,30,1.0,3.0,2.0,1,3.0,2.0,1
30,12,31,3.0,5.0,2.0,1,2.0,-1.0,-1
71,12,48,5.0,5.0,0.0,0,4.0,-1.0,-1
92,12,49,5.0,0.0,-5.0,-1,0.0,-5.0,-1
169,12,92,0.0,0.0,0.0,0,0.0,0.0,0
177,12,93,0.0,0.0,0.0,0,0.0,0.0,0
235,12,127,0.0,1.0,1.0,1,0.0,0.0,0
240,12,128,1.0,0.0,-1.0,-1,0.0,-1.0,-1
309,12,169,0.0,7.0,7.0,1,1.0,1.0,1


In [16]:
df = ret['pitmodel-TIMEDIFF-2018-outlap-nopitage'][0]
df[df['carno']==12]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
15,12,30,1.0,3.0,2.0,1,4.0,3.0,1
29,12,31,3.0,5.0,2.0,1,2.0,-1.0,-1
69,12,48,5.0,5.0,0.0,0,3.0,-2.0,-1
90,12,49,5.0,0.0,-5.0,-1,3.0,-2.0,-1
166,12,92,0.0,0.0,0.0,0,0.0,0.0,0
174,12,93,0.0,0.0,0.0,0,0.0,0.0,0
230,12,127,0.0,1.0,1.0,1,0.0,0.0,0
235,12,128,1.0,0.0,-1.0,-1,0.0,-1.0,-1
304,12,169,0.0,7.0,7.0,1,2.0,2.0,1


In [19]:
stint._inlap_status = 0

stint.init('oracle')

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars


In [20]:
accx, retx = test_pitmodel('oracle', 'Indy500-2018', 'TIMEDIFF', loopcnt=1)
df_oracle_2018 = retx[0]

INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
model: acc={0.67}, mae={3.48}, rmse={5.20},r2={-0.52}, 
            naive: acc={0.10}, mae={4.16}, rmse={5.60},r2={0.58}
[[ 0.66666667  3.48427673  5.20038702 -0.51763358]
 [ 0.10062893  4.16352201  5.60211101  0.58482207]]


In [36]:
stint.get_evalret(df_oracle_2018)

model: acc={0.67}, mae={3.48}, rmse={5.20},r2={-0.52}, 
            naive: acc={0.10}, mae={4.16}, rmse={5.60},r2={0.58}


array([[ 0.66666667,  3.48427673,  5.20038702, -0.51763358],
       [ 0.10062893,  4.16352201,  5.60211101,  0.58482207]])

In [21]:
accx2, retx2 = test_pitmodel('oracle', 'Indy500-2019', 'TIMEDIFF', loopcnt=1)
df_oracle_2019 = retx2[0]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
11,12,31,3.0,5.0,2.0,1,2.0,-1.0,-1
38,12,49,5.0,0.0,-5.0,-1,3.0,-2.0,-1
86,12,93,0.0,1.0,1.0,1,1.0,1.0,1
117,12,128,1.0,7.0,6.0,1,0.0,-1.0,-1


In [53]:
def neweval(df_refer, mid):
    dfml = df_refer
    dfranknet = ret[mid][0]
    carlist = set(list(dfml['carno'].values))
    dfpits = []
    for car in carlist:
        pitlaps = dfml[dfml['carno']==car].startlap
        dfpits.append(dfranknet[(dfranknet['carno']==car) & (dfranknet['startlap'].isin(pitlaps))])

    df_ranknet = pd.concat(dfpits)
    accret = stint.get_evalret(df_ranknet)
    
    return df_ranknet, accret


def neweval2(df_refer, df2):
    dfml = df_refer
    dfranknet = df2
    carlist = set(list(dfml['carno'].values))
    dfpits = []
    for car in carlist:
        pitlaps = dfml[dfml['carno']==car].startlap
        dfpits.append(dfranknet[(dfranknet['carno']==car) & (dfranknet['startlap'].isin(pitlaps))])

    df_ranknet = pd.concat(dfpits)
    accret = stint.get_evalret(df_ranknet)
    
    return df_ranknet, accret

In [29]:
dfn, accn = {} ,{}

In [30]:
mid = 'oracle-TIMEDIFF-2018-inlap-pitage'
dfn[mid], accn[mid] = neweval(df_oracle_2018,mid)

model: acc={0.80}, mae={4.14}, rmse={6.47},r2={-0.97}, 
            naive: acc={0.12}, mae={6.31}, rmse={8.78},r2={-0.02}


In [31]:
mid = 'pitmodel-TIMEDIFF-2018-inlap-pitage'
dfn[mid], accn[mid] = neweval(df_oracle_2018,mid)

model: acc={0.73}, mae={5.41}, rmse={7.40},r2={-3.13}, 
            naive: acc={0.09}, mae={6.73}, rmse={9.11},r2={-0.08}


In [32]:
mid = 'pitmodel-RANK-2018-inlap-pitage'
dfn[mid], accn[mid] = neweval(df_oracle_2018,mid)

model: acc={0.66}, mae={6.10}, rmse={8.44},r2={-4.01}, 
            naive: acc={0.09}, mae={6.69}, rmse={9.08},r2={-0.08}


In [33]:
mid = 'pitmodel-RANK-2018-inlap-nopitage'
dfn[mid], accn[mid] = neweval(df_oracle_2018,mid)

model: acc={0.63}, mae={5.97}, rmse={8.34},r2={-14.20}, 
            naive: acc={0.09}, mae={6.73}, rmse={9.11},r2={-0.08}


In [34]:
mid = 'pitmodel-TIMEDIFF-2018-inlap-nopitage'
dfn[mid], accn[mid] = neweval(df_oracle_2018,mid)

model: acc={0.74}, mae={5.05}, rmse={7.18},r2={-1.75}, 
            naive: acc={0.09}, mae={6.69}, rmse={9.08},r2={-0.08}


In [39]:
mid = 'oracle-TIMEDIFF-2018-inlap-pitage'
dfn[mid][dfn[mid]['carno']==12]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
30,12,31,3.0,5.0,2.0,1,2.0,-1.0,-1
92,12,49,5.0,0.0,-5.0,-1,0.0,-5.0,-1
177,12,93,0.0,0.0,0.0,0,0.0,0.0,0
240,12,128,1.0,0.0,-1.0,-1,0.0,-1.0,-1


In [38]:
df_oracle_2018[df_oracle_2018['carno']==12]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
11,12,31,3.0,5.0,2.0,1,2.0,-1.0,-1
38,12,49,5.0,0.0,-5.0,-1,3.0,-2.0,-1
86,12,93,0.0,1.0,1.0,1,1.0,1.0,1
117,12,128,1.0,7.0,6.0,1,0.0,-1.0,-1


In [40]:
acc2, ret2 = {},{}

In [41]:
#stint.FEATURE_PITAGE, stint.FEATURE_STATUS
mid = 'oracle-timediff-2018-inlap-nopitage'
acc2[mid], ret2[mid] = test_pitmodel('oracle', 'Indy500-2018', 'TIMEDIFF', loopcnt=1,
                                     inlapmode=1,featuremode = stint.FEATURE_STATUS)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
model: acc={0.76}, mae={3.25}, rmse={4.96},r2={0.23}, 
            naive: acc={0.11}, mae={5.25}, rmse={7.11},r2={0.45}
[[0.75524476 3.24825175 4.95539545 0.22797953]
 [0.11188811 5.24825175 7.10977855 0.4506735 ]]


In [65]:
#stint.FEATURE_PITAGE, stint.FEATURE_STATUS
mid = 'pitmodel-rank-2018-inlap-nopitage'
acc2[mid], ret2[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'RANK', loopcnt=1,
                                     inlapmode=1,featuremode = stint.FEATURE_STATUS)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p1.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-45 not found in model
model: acc={0.54}, mae={4.35}, rmse={5.81},r2={-4.72}, 
            naive: acc={0.09}, mae={4.42}, rmse={5.76},r2={0.56}
[[ 0.54109589  4.34931507  5.80918026 -4.72282552]
 [ 0.0890411   4.42465753  5.75528346  0.56131122]]


In [68]:
#stint.FEATURE_PITAGE, stint.FEATURE_STATUS
mid = 'pitmodel-rank-2018-noinlap-nopitage'
acc2[mid], ret2[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'RANK', loopcnt=1,
                                     inlapmode=0,featuremode = stint.FEATURE_STATUS)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
ERROR: key 14-45 not found in model
model: acc={0.53}, mae={4.14}, rmse={5.59},r2={-3.54}, 
            naive: acc={0.08}, mae={4.25}, rmse={5.52},r2={0.61}
[[ 0.53061224  4.13605442  5.59275527 -3.54347826]
 [ 0.08163265  4.25170068  5.51620988  0.60545236]]


In [74]:
#stint.FEATURE_PITAGE, stint.FEATURE_STATUS
mid = 'pitmodel-rank-2018-noinlap-pitage'
acc2[mid], ret2[mid] = test_pitmodel('pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle', 'Indy500-2018', 'RANK', loopcnt=1,
                                     inlapmode=0,featuremode = stint.FEATURE_PITAGE)

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['rank_diff'][mask] = 0
/N/u/pengb/hpda/indycar/predictor/src/indycar/model/stint_simulator_shortterm_pitmodel.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_ds['time_diff'][mask] = 0
INFO:root:Using GPU


init: load dataset laptime_rank_timediff_pit-oracle-Indy500_2013_2019_v9_p0.pickle with 7 races, 58 cars
init model:pitmodel-m65-mlp-dsel-e500-l10-10-5-student-d0.1.pickle
init pitmodel as PitModelMLP(pitmodel)
predicting model=oracle, plen=2
loading model...done!, ctx:gpu(0)
start pitlap: 28
--------- start update_onets: 28 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
--------- after update_onets: 28 ----------
[0. 0.

--------- simu_onestep: 29-57, endpos=42: 29 ----------
[2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 4.]
[4.03042507 3.86476684 2.92397428 3.72026181 3.78652072 3.53466105
 3.50155401 3.50515127 3.50295186 3.50673604 6.         6.        ]
--------- simu_onestep: 29-57, endpos=44: 29 ----------
[2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 4.]
[4.03042507 3.86476684 2.92397428 3.72026181 3.78652072 3.53466105
 3.50155401 3.50515127 3.50295186 3.50673604 3.50535655 3.50315571
 6.         6.        ]
--------- simu_onestep: 29-57, endpos=46: 29 ----------
[2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 4.]
[4.03042507 3.86476684 2.92397428 3.72026181 3.78652072 3.53466105
 3.50155401 3.50515127 3.50295186 3.50673604 3.50535655 3.50315571
 3.49962473 3.49499846 6.         6.        ]
--------- simu_onestep: 29-57, endpos=48: 29 ----------
[2. 2. 3. 3. 3. 3. 3. 3.

--------- simu_onestep: 30-61, endpos=59: 30 ----------
[2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 4. 2.]
[1.88538313 2.72079349 4.32044077 3.94145751 3.79752541 3.72525907
 3.73984098 3.7555027  3.75889039 3.76332831 3.7610836  3.7605021
 3.76277709 3.76468253 3.76328373 3.76305366 3.76016259 3.75895286
 3.75759768 3.75577664 3.75396252 3.75369024 3.75280094 3.75295186
 3.750386   3.75340581 6.         6.        ]
--------- simu_onestep: 30-61, endpos=61: 30 ----------
[2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 4. 2.]
[1.88538313 2.72079349 4.32044077 3.94145751 3.79752541 3.72525907
 3.73984098 3.7555027  3.75889039 3.76332831 3.7610836  3.7605021
 3.76277709 3.76468253 3.76328373 3.76305366 3.76016259 3.75895286
 3.75759768 3.75577664 3.75396252 3.75369024 3.75280094 3.75295186
 3.750386   3.75340581 3.75860286 3.76422191 6.         6.        ]
--------- simu_onestep: 30-61, endpos=63: 30 ---

--------- simu_onestep: 31-64, endpos=66: 31 ----------
[2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 4. 2. 4.]
[4.85078478 4.72117281 4.712852   4.6696825  4.66371441 4.66157293
 4.69205952 4.71489763 4.72522354 4.73340607 4.73985195 4.74449301
 4.74439144 4.74810839 4.75025749 4.74439096 4.74276114 4.74105787
 4.74196482 4.7407136  4.74386883 4.7449975  4.74783182 4.75081015
 4.7546916  4.76677847 4.76932144 5.31481123 5.10913515 5.12367058
 5.19552612 5.21658802 5.         5.        ]
simulation done: 33
start pitlap: 32
--------- start update_onets: 32 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

...

simulation done: 25
start pitlap: 179
--------- start update_onets: 179 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------- after update_onets: 179 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

simulation done: 25
start pitlap: 184
--------- start update_onets: 184 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------- after update_onets: 184 ----------
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

--------- simu_onestep: 186-198, endpos=193: 186 ----------
[ 2.  2.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  4.  2.  4. 18. 10.  6.  6.
  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  4.  4.  4.  4.
  4.  6.  6.  6.  6.  6.  6.  6.  6.  5.  5.  5.  5.  5.  5.  4.  4.  4.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  2.
  2.  1.  1.  1. 13.  9.  8.  8.  8.  8.  6.  6.  6.  5.  5.  2.  2.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  2. 11.  7.  6.  6.  5.  4.  4.  3.  2.  2.  2.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  8. 16. 13. 10.  9.  8.  7.  7.  7.  6.
  6.  5.  5.  5.  4.  4.  4.]
[3.99610329 3.9651587  3.90275383 3.89485312 4.         4.        ]
--------- simu_onestep: 186-198, endpos=195: 186 ----------
[ 2.  2.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.

--------- simu_onestep: 189-199, endpos=200: 189 ----------
[ 2.  2.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  4.  2.  4. 18. 10.  6.  6.
  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  4.  4.  4.  4.
  4.  6.  6.  6.  6.  6.  6.  6.  6.  5.  5.  5.  5.  5.  5.  4.  4.  4.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  2.
  2.  1.  1.  1. 13.  9.  8.  8.  8.  8.  6.  6.  6.  5.  5.  2.  2.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  2. 11.  7.  6.  6.  5.  4.  4.  3.  2.  2.  2.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  8. 16. 13. 10.  9.  8.  7.  7.  7.  6.
  6.  5.  5.  5.  4.  4.  4.  4.  4.  4.]
[3.97280574 3.94591188 3.95107245 3.94853425 3.9453907  3.94602871
 3.95705533 3.97328091 1.         1.        ]
simulation done: 24
start pitlap: 190
--------- start u

--------- simu_onestep: 195-198, endpos=200: 195 ----------
[ 2.  2.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  4.  2.  4. 18. 10.  6.  6.
  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  4.  4.  4.  4.
  4.  6.  6.  6.  6.  6.  6.  6.  6.  5.  5.  5.  5.  5.  5.  4.  4.  4.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  2.
  2.  1.  1.  1. 13.  9.  8.  8.  8.  8.  6.  6.  6.  5.  5.  2.  2.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  2. 11.  7.  6.  6.  5.  4.  4.  3.  2.  2.  2.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  8. 16. 13. 10.  9.  8.  7.  7.  7.  6.
  6.  5.  5.  5.  4.  4.  4.  4.  4.  4.  4.  4.  4.  3.  3.  1.]
[0.97049963 0.9541533  1.         1.        ]
simulation done: 23
start pitlap: 196
--------- start update_onets: 196 ----------
[0. 0. 0. 0. 0.

In [75]:
df= ret2[mid][0]
df[df['carno']==12]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
10,12,31,3.0,5.0,2.0,1,1.0,-2.0,-1
36,12,49,5.0,0.0,-5.0,-1,4.0,-1.0,-1
84,12,93,0.0,1.0,1.0,1,0.0,0.0,0
113,12,128,1.0,7.0,6.0,1,0.0,-1.0,-1


In [76]:
df

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
0,13,28,20.0,12.0,-8.0,-1,16.0,-4.0,-1
1,14,29,10.0,1.0,-9.0,-1,6.0,-4.0,-1
2,59,29,32.0,22.0,-10.0,-1,27.0,-5.0,-1
3,3,30,10.0,4.0,-6.0,-1,7.0,-3.0,-1
4,20,30,7.0,0.0,-7.0,-1,0.0,-7.0,-1
...,...,...,...,...,...,...,...,...,...
143,64,156,19.0,14.0,-5.0,-1,16.0,-3.0,-1
144,59,157,23.0,19.0,-4.0,-1,20.0,-3.0,-1
145,60,157,19.0,17.0,-2.0,-1,13.0,-6.0,-1
146,23,159,19.0,18.0,-1.0,-1,18.0,-1.0,-1


In [55]:
neweval2(df_oracle_2018,ret2[mid][0])

model: acc={0.63}, mae={4.05}, rmse={5.85},r2={-0.72}, 
            naive: acc={0.10}, mae={4.40}, rmse={5.89},r2={0.54}


(     carno  startlap  startrank  endrank  diff  sign  pred_endrank  pred_diff  \
 25       1        31        2.0      2.0   0.0     0           3.0        1.0   
 65       1        49        2.0      8.0   6.0     1          10.0        8.0   
 111      1        69        8.0      8.0   0.0     0          12.0        4.0   
 167      1       100        8.0      7.0  -1.0    -1          13.0        5.0   
 212      1       134        7.0     16.0   9.0     1           7.0        0.0   
 ..     ...       ...        ...      ...   ...   ...           ...        ...   
 260     88       170       16.0     17.0   1.0     1          18.0        2.0   
 36      98        31        6.0      9.0   3.0     1          11.0        5.0   
 84      98        49        9.0     14.0   5.0     1          11.0        2.0   
 132     98        91       14.0      9.0  -5.0    -1          11.0       -3.0   
 186     98       124        9.0     13.0   4.0     1           7.0       -2.0   
 
      pred_sig

In [56]:
neweval2(df_oracle_2018,df_oracle_2018)

model: acc={0.67}, mae={3.48}, rmse={5.20},r2={-0.52}, 
            naive: acc={0.10}, mae={4.16}, rmse={5.60},r2={0.58}


(     carno  startlap  startrank  endrank  diff  sign  pred_endrank  pred_diff  \
 9        1        31        2.0      2.0   0.0     0           3.0        1.0   
 33       1        49        2.0      8.0   6.0     1           9.0        7.0   
 67       1        69        8.0      8.0   0.0     0           6.0       -2.0   
 99       1       100        8.0      7.0  -1.0    -1          13.0        5.0   
 127      1       134        7.0     16.0   9.0     1           7.0        0.0   
 ..     ...       ...        ...      ...   ...   ...           ...        ...   
 151     88       170       16.0     17.0   1.0     1          18.0        2.0   
 16      98        31        6.0      9.0   3.0     1          10.0        4.0   
 52      98        49        9.0     14.0   5.0     1          12.0        3.0   
 80      98        91       14.0      9.0  -5.0    -1          11.0       -3.0   
 112     98       124        9.0     13.0   4.0     1           6.0       -3.0   
 
      pred_sig

In [72]:
df_oracle_2018[140:]

,carno,startlap,startrank,endrank,diff,sign,pred_endrank,pred_diff,pred_sign
140,14,142,21.0,9.0,-12.0,-1,18.0,-3.0,-1
141,26,149,24.0,23.0,-1.0,-1,23.0,-1.0,-1
142,19,151,20.0,15.0,-5.0,-1,12.0,-8.0,-1
143,26,155,23.0,22.0,-1.0,-1,23.0,0.0,0
144,59,155,21.0,23.0,2.0,1,22.0,1.0,1
145,23,156,16.0,19.0,3.0,1,16.0,0.0,0
146,25,156,18.0,16.0,-2.0,-1,16.0,-2.0,-1
147,64,156,19.0,14.0,-5.0,-1,15.0,-4.0,-1
148,59,157,23.0,19.0,-4.0,-1,21.0,-2.0,-1
149,60,157,19.0,17.0,-2.0,-1,17.0,-2.0,-1
